# Лабораторная работа 1
### Выполнил: ФИО

Добро пожаловать в лабораторную работу! 
Оформите ваше решение контеста Kaggle в соответствии с этим шаблоном. Ваша задача - показать проверяющим ход ваших рассуждений, поэтому советуем писать много комментариев к коду и приводить комментарии к логике на каждом этапе в текстовых ячейках. 
Где необходимо, используйте графики для большей наглядности.

При отправке поменяйте название файла на ваши ФИО!

### EDA (исследовательский анализ данных)

В этом разделе вам необходимо провести анализ вашего датасета, интерпретировать признаки, выяснить их значимость и исследовать зависимости между ними.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
features = pd.read_csv('data/DOTA2_TRAIN_features.csv')
targets = pd.read_csv('data/DOTA2_TRAIN_targets.csv')
df = pd.merge(targets, features, on='match_id')

In [3]:
test = pd.read_csv('data/DOTA2_TEST_features.csv')

In [4]:
original_df = pd.merge(features.dropna().copy(), targets, on='match_id')
original_features = original_df.drop(columns=['radiant_win', 'match_id'], axis=1)
original_targets = original_df.radiant_win

разбил данные на две части: в одной буду заменять наны на что-то, а в другой просто все удалю
посмотрим на уникальные значения в каждом столбце и сделаем вывод о том, какие призкгнаки категориальные

In [5]:
categorial_features = ['lobby_type', 'game_mode', 'r5_firstblood_claimed', 'r1_firstblood_claimed',
                       'r2_firstblood_claimed', 'r3_firstblood_claimed', 'r4_firstblood_claimed',
                       'd5_firstblood_claimed', 'd1_firstblood_claimed', 'd2_firstblood_claimed',
                       'd3_firstblood_claimed', 'd4_firstblood_claimed']

In [6]:
original_features.nunique()

game_time                     3051
game_mode                        7
lobby_type                       2
objectives_len                  39
chat_len                       136
                              ... 
d5_teamfight_participation     575
d5_towers_killed                 9
d5_roshans_killed                5
d5_obs_placed                   24
d5_sen_placed                   36
Length: 245, dtype: int64

**Делаем бэйслайн с замененными значениями и удаленными**


In [7]:
df.fillna(-999, inplace=True)

### Preprocessing (подготовка данных)

 В этом разделе вам необходимо реализовать подготовку ваших данных, в том числе заполнение пропусков, фильтрацию выбросов, кодирование категориальных признаков и т.д. В этот же раздел включите любые операции над данными, которые сочтете нужными.

In [8]:
# preprocessing pipeline

### Model & training (Выбор модели и её обучение)

В этом разделе описываете модель и ставите эксперименты по обучению.

Если вы ставили много экспериментов, приведите их в хронологическом порядке чтобы мы увидели эволюцию ваших рассуждений.

#### Эксперимент 1

In [9]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import roc_auc_score
X = original_features
y = original_targets

X_test = test.drop(columns='match_id', axis=1)
test_index = test.match_id

In [10]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

In [11]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [12]:
model.fit(
    X_train, y_train,
    cat_features=categorial_features,
    eval_set=(X_validation, y_validation),
    #     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [13]:
y = model.predict(X_validation)
roc_auc_score(y_validation, y)

0.6994061807795856

ro

In [14]:
predit_proba = model.predict_proba(X_test)

In [15]:
outer = pd.Series(data = predit_proba[:, 1] , index=test_index)
outer.name= 'radiant_win'
outer.to_csv('predictions/predictions1.csv')

#### Эксперимент 2

In [16]:
X = df.drop(columns= ['match_id', 'radiant_win'], axis=1)
y = df.radiant_win
X_test = test.drop(columns='match_id', axis=1)
test_index = test.match_id
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)
model.fit(
    X_train, y_train,
    cat_features=categorial_features,
    eval_set=(X_validation, y_validation),
    #     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [17]:
y = model.predict(X_validation)
roc_auc_score(y_validation, y)


0.7097083913916633

**Вывод**
если заменить пропущенные значения на отельный класс то результат будет лучше

In [18]:
predit_proba = model.predict_proba(X_test)

outer = pd.Series(data=predit_proba[:, 1], index=test_index)
outer.name = 'radiant_win'
outer.to_csv('predictions/predictions2.csv')

#### Эксперимент 3
попробуем кроссвалидацию

In [19]:
X = df.drop(columns= ['match_id', 'radiant_win'], axis=1)
y = df.radiant_win
X_test = test.drop(columns='match_id', axis=1)
test_index = test.match_id
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)
model.fit(
    X_train, y_train,
    cat_features=categorial_features,
    eval_set=(X_validation, y_validation),
    #     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [20]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorial_features),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [21]:
y = model.predict(X_validation)
roc_auc_score(y_validation, y)


0.7097083913916633

### Эксперимент 4
попробуем уменьшить тестовую выборку


In [22]:
X = df.drop(columns= ['match_id', 'radiant_win'], axis=1)
y = df.radiant_win
X_test = test.drop(columns='match_id', axis=1)
test_index = test.match_id
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)
model.fit(
    X_train, y_train,
    cat_features=categorial_features,
    eval_set=(X_validation, y_validation),
    #     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Модель начала переобучаться 

In [23]:
y = model.predict(X_validation)
roc_auc_score(y_validation, y)

0.7173347232217868

In [24]:
predit_proba = model.predict_proba(X_test)

outer = pd.Series(data=predit_proba[:, 1], index=test_index)
outer.name = 'radiant_win'
outer.to_csv('predictions/predictions3.csv')

### Эксперимент 5

### Evaluation (оценка качества модели)

В этом разделе проводите оценку качества вашей итоговой модели. 

In [25]:
# model evaluation

### Conclusion (Выводы)

В этом разделе описываете полученные результаты и проводите анализ выполненной работы. 
Что получилось / не получилось и почему?